In [27]:
import collections
import string
import itertools
import re

In [28]:
Link = collections.namedtuple('Link', 'height left right')

In [29]:
def extract_pairs(net_string):
    return [[int(pi) for pi in p.split(', ')] for p in net_string[1:-1].split('), (')]

In [30]:
def read_net_string(net_string):
    return [Link(h, l, r) for h, (l, r) in enumerate(extract_pairs(net_string))]

In [31]:
def read_net(filename):
    with open(filename) as f:
        pairs = [re.split('\D+', p.strip()) for p in f.readlines()]
        lrs = [(int(lr[1]), int(lr[2])) for lr in pairs]
        return [Link(h, l, r) 
                for h, (l, r) in enumerate(lrs)]

In [32]:
small_net = read_net('04-small.txt')
small_net

[Link(height=0, left=2, right=5),
 Link(height=1, left=1, right=4),
 Link(height=2, left=0, right=3),
 Link(height=3, left=0, right=3),
 Link(height=4, left=0, right=5),
 Link(height=5, left=3, right=5),
 Link(height=6, left=0, right=2),
 Link(height=7, left=3, right=4),
 Link(height=8, left=2, right=4),
 Link(height=9, left=1, right=2),
 Link(height=10, left=0, right=4),
 Link(height=11, left=1, right=2),
 Link(height=12, left=2, right=4),
 Link(height=13, left=0, right=4),
 Link(height=14, left=1, right=4)]

In [33]:
net = read_net('04-lines.txt')
len(net)

10135

In [34]:
def show_net(links, pair_sep=', '):
    return pair_sep.join('({}, {})'.format(l.left, l.right) for l in sorted(links))

In [35]:
def link_ends(link):
    return set((link.left, link.right))

In [36]:
def follow(initial_line, links):
    line = initial_line
    heights = sorted(set(l.height for l in links))
    for h in heights:
        for l in [l for l in links if l.height == h]:
            if line in link_ends(l):
                line = [e for e in link_ends(l) if e != line][0]
#                 print(l, line)
    return line

In [37]:
def pack(net):
    packed_links = []
    line_heights = collections.defaultdict(lambda: -1)
    for link in sorted(net):
        link_height = max(line_heights[link.left], line_heights[link.right]) + 1
        line_heights[link.left] = link_height
        line_heights[link.right] = link_height
        packed_links += [Link(link_height, link.left, link.right)]
    return sorted(packed_links)

In [38]:
max(l.height for l in small_net)

14

In [39]:
max(l.height for l in pack(small_net))

10

In [40]:
max(l.height for l in net)

10134

In [41]:
pnet = pack(net)

In [42]:
max(l.height for l in pnet)

2286

In [43]:
def height_groups(net):
    return {h: list(g) for h, g in itertools.groupby(pack(net), lambda l: l.height)}

In [44]:
def follow_many(in_sequence, net):
    hgs = height_groups(net)
    seq = list(in_sequence)
    for h in hgs:
        for link in hgs[h]:
            seq[link.right], seq[link.left] = seq[link.left], seq[link.right]
    return seq

In [45]:
''.join(follow_many('abcdef', small_net))

'acfbed'

In [46]:
%%timeit
follow_many('abcdefghij', small_net)

10000 loops, best of 3: 39.5 µs per loop


In [47]:
''.join(follow_many(string.ascii_lowercase, net))

'doqzmbishkwunvltpcexyjgfra'

In [48]:
%%timeit
follow_many(string.ascii_lowercase, net)

10 loops, best of 3: 19.7 ms per loop


In [49]:
%%timeit
follow_many(string.ascii_lowercase, pnet)

100 loops, best of 3: 18.6 ms per loop
